In [ ]:
# import all necessary packages
import numpy as np

import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
%matplotlib inline

import scipy.linalg

from paths import add_paths_to_sys
add_paths_to_sys()

import datafold.pcfold as pfold
import datafold.dynfold as dfold


# PCMKoopman

Demonstrates how to use the PCMKoopman class.

In [ ]:
# setup the example

np.random.seed(2)

n_pts = 1000
n_centers = 150

# create random contraction
A = np.random.randn(2, 2)
A = A @ A.T
u, _, v = np.linalg.svd(A)
A = u @ (np.diag([.7, .5])) @ v.T

print(np.linalg.eigvals(A))
A = scipy.linalg.fractional_matrix_power(A, .25)
print(A)

base_data = 1.2 * (np.random.rand(n_centers, 2) - .5)
x_data = np.random.rand(n_pts, 2) - .5
y_data = x_data @ A

fig, ax = plt.subplots(1, 1, figsize=(15, 15))
ax = [ax]
ax[0].scatter(*x_data.T, s=1, c='black')
ax[0].scatter(*y_data.T, s=1, c='red')
ax[0].scatter(*base_data.T, s=10, c='green')

In [ ]:
# setup the manifold

pcm = pfold.PCManifold(base_data, dist_backend="scipy.kdtree")
pcm.optimize_parameters(result_scaling=3)

# this is to align to previous results which uses another equation for the Gaussian kernel
pcm.kernel.epsilon = 0.5 * pcm.kernel.epsilon**2

# construct the operator
pcm_edmd = dfold.PCMKoopman(pcm)
pcm_edmd.build(x_data, y_data, regularizer_strength=1e-6)

In [ ]:
# try prediction
np.random.seed(1)
x_c = (np.random.rand(n_pts, 2) - .5) / 1.5
x_t = []
x_t_true = x_c
NT = 15

fig, ax = plt.subplots(1, NT, figsize=(12, 3), sharex=True, sharey=True)
x_t = pcm_edmd.predict(x_c, NT=NT)

print(x_t[0].shape)
for k in range(NT):
    ax[k].scatter(*x_t_true.T, s=3, c='black')
    ax[k].scatter(*x_t[(k * 1) % len(x_t)].T, s=.5, c='red')

    x_t_true = x_t_true @ A

ax[0].scatter(*x_t[0].T, s=1, c='black')